# Data Analysis of CEAPS

In [1]:
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display

py.init_notebook_mode(connected=True)

In [2]:
# Chart default layout
default_layout = dict(
    titlefont=dict(size=18, color="darkblue"),
    tickfont=dict(size=14, color="black"),
    showgrid=True,
    zeroline=True,
    showline=True,
    mirror=True,
    gridcolor="lightgrey",
    gridwidth=1,
    zerolinecolor="grey",
    zerolinewidth=2,
    linecolor="black",
    linewidth=2,
)

In [3]:
# Reading the data
original_df = pd.read_csv("../data/data.csv")
original_df.head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2014,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ELETOBRAS DISTRIBUIÇĂO RONDÔNIA,452163-3,2014-01-10,Despesa com pagamento de Energia Elétrica para...,52.38,869442.0
1,2014,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ELETROBRAS DISTRIBUIÇĂO RONDÔNIA,000060693,2014-01-18,Despesa com pagamento de Energia Elétrica para...,158.26,869445.0
2,2014,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,001/14,2014-01-31,Despesa com pagamento de aluguel de imóvel par...,5000.00,869452.0
3,2014,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.423.963/0001-11,OI MÓVEL S.A.,493020122,2014-01-20,Despesa com pagamento de telefones fixo e móve...,401.71,869446.0
4,2014,1,ACIR GURGACZ,Divulgaçăo da atividade parlamentar,84.721.745/0001-30,ALERTA COMUNICAÇĂO O. F. CALADO EDIÇƠES,000127,2014-01-20,Despesa com material impresso para divulgaçăo ...,726.88,896991.0



## Query the data

### Choose the senator

In [4]:
senator = widgets.Dropdown(
    options=original_df["SENADOR"].unique().tolist() + ["All"],
    value="All",
    description="Senator:",
    disabled=False,
)
display(senator)

Dropdown(description='Senator:', index=291, options=('ACIR GURGACZ', 'AÉCIO NEVES', 'ALFREDO NASCIMENTO', 'ALO…

### Choose the supplier

TODO: the dropdown was not working. Require fix.

### Choose the year

In [5]:
range_year = widgets.IntRangeSlider(
    value=[2010, 2022], min=original_df["ANO"].min(), max=original_df["ANO"].max(), step=1, description="Years"
)
display(range_year)

IntRangeSlider(value=(2010, 2022), description='Years', max=2022, min=2008)

### **After setting the valus above, run the cell below and the others!**

In [22]:
# Querying the data

if senator.value == "All":
    df = original_df[original_df["ANO"].isin(range(range_year.value[0], range_year.value[1] + 1))]
else:
    df = original_df[original_df["SENADOR"] == senator.value]
    
print(
    f"Getting data from {range_year.value[0]} to {range_year.value[1]} for {senator.value}.\n"
)
print(df['ANO'].value_counts().sort_index())

Getting data from 2010 to 2022 for AÉCIO NEVES.

ANO
2013    361
2016    307
2012    303
2015    264
2017    231
2018    186
2011    164
2014    152
2019     15
Name: count, dtype: int64


## Reibursement by year

In [7]:
# Define x and y axis layout
xaxis_layout = default_layout.copy()
xaxis_layout["title"] = "Year"

yaxis_layout = default_layout.copy()
yaxis_layout["title"] = "Total Reimbursement (R$)"

In [8]:
# Define data
group_by_year = df.groupby("ANO")

# Sum of values by year
reimbursement_by_year = group_by_year["VALOR_REEMBOLSADO"].sum().reset_index()
reimbursement_by_year

,ANO,VALOR_REEMBOLSADO
0,2010,10760010.57
1,2011,19004502.48
2,2012,23354742.37
3,2013,24658952.86
4,2014,22519394.12
5,2015,28893568.61
6,2016,25301959.82
7,2017,26673327.06
8,2018,25604701.33
9,2019,25248242.40


In [9]:
# Plot the data
trace = go.Scatter(
    x=reimbursement_by_year["ANO"],
    y=reimbursement_by_year["VALOR_REEMBOLSADO"],
    mode="markers+lines",
    marker=dict(size=10, color="blue", symbol="circle"),
    line=dict(width=2, color="blue"),
)

# Set chart layout
layout = go.Layout(
    title="Reimbursement by Year",
    titlefont=dict(size=24, color="darkblue"),
    xaxis=xaxis_layout,
    yaxis=yaxis_layout,
    hovermode="closest",
    plot_bgcolor="white",
)

py.iplot({"data": [trace], "layout": layout})

## Distribution of Types of Expenses

In [10]:
# Define x and y axis layout
xaxis_layout = default_layout.copy()
# xaxis_layout["title"] = "Type of expense"

yaxis_layout = default_layout.copy()
yaxis_layout["title"] = "Quantity"

In [11]:
# Define the data
df["TIPO_DESPESA"] = df["TIPO_DESPESA"].astype(str)
count_by_type = df["TIPO_DESPESA"].value_counts().reset_index()
count_by_type.columns = ["TIPO_DESPESA", "COUNT"]
count_by_type

/tmp/ipykernel_26925/1618930082.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,TIPO_DESPESA,COUNT
0,"Locomoçăo, hospedagem, alimentaçăo, combustíve...",122186
1,"Passagens aéreas, aquáticas e terrestres nacio...",68763
2,"Aluguel de imóveis para escritório político, c...",58968
3,Aquisiçăo de material de consumo para uso no e...,23639
4,Divulgaçăo da atividade parlamentar,12722
5,"Contrataçăo de consultorias, assessorias, pesq...",11835
6,Serviços de Segurança Privada,944


In [12]:
# Plot the data
trace = go.Bar(
    x=count_by_type["TIPO_DESPESA"],
    y=count_by_type["COUNT"],
    marker=dict(color="blue"),
    text=count_by_type["COUNT"],
    textposition="auto",
)

# Set chart layout
layout = go.Layout(
    title="Amount of expenses by type",
    titlefont=dict(size=24, color="darkblue"),
    xaxis=dict(
        xaxis_layout,
        showticklabels=False,  # Turn off X axis values
    ),
    yaxis=yaxis_layout,
    hovermode="closest",
    plot_bgcolor="white",
)

py.iplot({"data": [trace], "layout": layout})

## Sum of the expenses by type

In [13]:
# Define x and y axis layout
xaxis_layout = default_layout.copy()
# xaxis_layout["title"] = "Type of expense"

yaxis_layout = default_layout.copy()
yaxis_layout["title"] = "Amount reimbursed (R$)"

In [14]:
# Define the data
amount_by_type = (
    df.groupby("TIPO_DESPESA")["VALOR_REEMBOLSADO"]
    .sum()
    .reset_index()
    .sort_values("VALOR_REEMBOLSADO", ascending=False)
)
amount_by_type

,TIPO_DESPESA,VALOR_REEMBOLSADO
5,"Passagens aéreas, aquáticas e terrestres nacio...",76059445.64
4,"Locomoçăo, hospedagem, alimentaçăo, combustíve...",67702225.63
0,"Aluguel de imóveis para escritório político, c...",55149657.99
2,"Contrataçăo de consultorias, assessorias, pesq...",51983254.59
3,Divulgaçăo da atividade parlamentar,39418784.84
1,Aquisiçăo de material de consumo para uso no e...,11372238.20
6,Serviços de Segurança Privada,3092528.50


In [15]:
# Plot the data
trace = go.Bar(
    x=amount_by_type["TIPO_DESPESA"],
    y=amount_by_type["VALOR_REEMBOLSADO"],
    marker=dict(color="blue"),
    text=amount_by_type["VALOR_REEMBOLSADO"],
    textposition="auto",
)

# Set chart layout
layout = go.Layout(
    title="Amount reimbursed by type of expense",
    titlefont=dict(size=24, color="darkblue"),
    xaxis=dict(
        xaxis_layout,
        showticklabels=False,  # Turn off X axis values
    ),
    yaxis=yaxis_layout,
    hovermode="closest",
    plot_bgcolor="white",
)

py.iplot({"data": [trace], "layout": layout})

## Suppliers of the services (top 20)

In [16]:
# Define x and y axis layout
xaxis_layout = default_layout.copy()
xaxis_layout["title"] = "Supplier"

yaxis_layout = default_layout.copy()
yaxis_layout["title"] = "Amount paid (R$)"

In [17]:
# Define the data
amount_by_supplier = (
    df.groupby("FORNECEDOR")["VALOR_REEMBOLSADO"]
    .sum()
    .reset_index()
    .sort_values("VALOR_REEMBOLSADO", ascending=False)
)
amount_by_supplier[:20]

,FORNECEDOR,VALOR_REEMBOLSADO
26018,LATAM,5276803.89
19893,GOL,4652758.73
45402,TAM,4507334.20
35712,PROPAGTUR - Propag Turismo Ltda,4442760.27
936,ADRIA VIAGENS E TURISMO LTDA,3450669.70
12883,Citel Service,1634467.13
24960,Jefferson L R Coronel - Me,1520200.00
934,ADRIA VIAGENS E TURISMO,1513759.85
4331,Adria Viagens e Turismo Ltda,1473108.67
3914,AVIANCA,1363992.97


In [ ]:
# Plot the data
trace = go.Bar(
    x=amount_by_supplier["FORNECEDOR"][:20],
    y=amount_by_supplier["VALOR_REEMBOLSADO"][:20],
    marker=dict(color="blue"),
    text=amount_by_supplier["VALOR_REEMBOLSADO"][:20],
    textposition="auto",
)

# Set chart layout
layout = go.Layout(
    title="Amount reimbursed by supplier",
    titlefont=dict(size=24, color="darkblue"),
    xaxis=dict(
        xaxis_layout,
        showticklabels=False,  # Turn off X axis values
    ),
    yaxis=yaxis_layout,
    hovermode="closest",
    plot_bgcolor="white",
)

py.iplot({"data": [trace], "layout": layout})